In [1]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv("key.env",verbose=True)

True

# Language Models

### Completions API (Deprecated)

#### LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)


result = llm("自己紹介してください")
print(result)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


### Chat Completions API

#### Chat models

- AIMessage
- HumanMessage
- SystemMessage

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？")
]
result = chat(messages)
print(result.content)


はい、おっしゃった通り、あなたの名前はジョンさんですね。他に何かお手伝いできることがありますか？


#### Streaming with Callbacks

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages) # type:ignore


はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問や会話に対応することができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。

# Prompts

## PromptTemplate

In [6]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください

料理名: {dish}
"""

prompt = PromptTemplate(
    input_variables=["dish"],
    template=template
)

result = prompt.format(dish="カレー")
print(result)


以下の料理のレシピを考えてください

料理名: カレー



## ChatPromptTemplate

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
        HumanMessagePromptTemplate.from_template("以下の料理のレシピを考えてください。\n\n料理名: {dish}"),
    ]
)

messages = chat_prompt.format_prompt(country="イギリス", dish="肉じゃが").to_messages()
pprint(messages)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。', additional_kwargs={}),
 HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが', additional_kwargs={}, example=False)]


# Output parsers
LLMの出力からPythonオブジェクトを作成する

## PydanticOutputParser

In [3]:
from pydantic import BaseModel, Field
from typing import List

class Recipe(BaseModel):
    ingredients: List[str] = Field(
        description="ingredients of the dish",
    )
    steps: List[str] = Field(
        description="steps to make the dish",
    )

In [4]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [5]:
format_instructions = parser.get_format_instructions()
pprint(format_instructions)

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"ingredients": {"description": "ingredients of the dish", '
 '"items": {"type": "string"}, "title": "Ingredients", "type": "array"}, '
 '"steps": {"description": "steps to make the dish", "items": {"type": '
 '"string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", '
 '"steps"]}\n'
 '```')


In [6]:
template = """
料理のレシピを考えてください

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": format_instructions}
)

In [7]:
formatted_prompt = prompt.format(dish="カレー")
pprint(formatted_prompt)

('\n'
 '料理のレシピを考えてください\n'
 '\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"ingredients": {"description": "ingredients of the dish", '
 '"items": {"type": "string"}, "title": "Ingredients", "type": "array"}, '
 '"steps": {"description": "steps to make the dish", "items": {"type": '
 '"string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", '
 '"steps"]}\n'
 '```\n'
 '\n'
 '料理名: カレー\n')


In [8]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = [HumanMessage(content=formatted_prompt)]
output = chat(messages)
pprint(output.content)

('```json\n'
 '{\n'
 '    "ingredients": [\n'
 '        "玉ねぎ",\n'
 '        "人参",\n'
 '        "じゃがいも",\n'
 '        "牛肉",\n'
 '        "カレールー",\n'
 '        "水"\n'
 '    ],\n'
 '    "steps": [\n'
 '        "1. 材料を準備する。",\n'
 '        "2. 玉ねぎ、人参、じゃがいもを切る。",\n'
 '        "3. 牛肉を炒める。",\n'
 '        "4. 野菜を加えて炒める。",\n'
 '        "5. 水を加えて煮込む。",\n'
 '        "6. カレールーを溶かして加える。",\n'
 '        "7. 全体を混ぜて完成。"\n'
 '    ]\n'
 '}\n'
 '```')


In [9]:
recipe = parser.parse(output.content)
print(type(recipe))
pprint(recipe)

<class '__main__.Recipe'>
Recipe(ingredients=['玉ねぎ', '人参', 'じゃがいも', '牛肉', 'カレールー', '水'], steps=['1. 材料を準備する。', '2. 玉ねぎ、人参、じゃがいもを切る。', '3. 牛肉を炒める。', '4. 野菜を加えて炒める。', '5. 水を加えて煮込む。', '6. カレールーを溶かして加える。', '7. 全体を混ぜて完成。'])


## Chains

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """
料理のレシピを考えてください

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
from langchain import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.run(dish="カレー")
print(type(recipe))
pprint(recipe)

<class '__main__.Recipe'>
Recipe(ingredients=['玉ねぎ', '人参', 'じゃがいも', '牛肉', 'カレールー', '水'], steps=['1. 材料を準備する。', '2. 玉ねぎ、人参、じゃがいもを切る。', '3. 牛肉を炒める。', '4. 野菜を加えて炒める。', '5. 水を加えて煮込む。', '6. カレールーを溶かして加える。', '7. 全体を混ぜて完成。'])


In [12]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

cot_template = """
以下の質問に回答してください

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [13]:
summarize_template = """
以下の文章を結論だけ一言に要約してください

{input}
"""

summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [14]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain],
)

result = cot_summarize_chain(
    "私は市場に行って10個のりんごを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのりんごを買って1つ食べました。残りは何個ですか？"
)

print(result["output"])

残りのりんごは10個。


## Memory

### ConversationBufferMemory

In [15]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True: # -> infinite loop
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")

AI: こんにちは、Tylerさん。私はAIです。どのようにお手伝いできますか？
AI: はい、Tylerさんの名前を知っています。それはあなたの自己紹介で教えてくれたからです。他に何か知りたいことがありますか？
AI: 何か質問があれば遠慮なくどうぞ。私はできる限りお手伝いします。
AI:  了解しました。もし何か他の質問やお手伝いが必要な場合はいつでもお知らせくださいね。良い一日を！
AI:  それでは、またお会いしましょう。良い一日を！
AI:  さようなら！またお会いしましょう。
AI:  さようなら！またお会いしましょう。
